In [1]:
import sys
import os
sys.path.append('../')
%load_ext autoreload
%autoreload 2

import dice_ml
import torch
from models.run_MLP import MLP
from utils.parser import *
from models.data_process import load_adult_income_dataset
from sklearn.model_selection import train_test_split

args = parse_args()
device = args.device

/Users/bytedance/miniconda3/envs/counterfactual/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = load_adult_income_dataset()

target = dataset["income"]
train_dataset, test_dataset, y_train, y_test = train_test_split(dataset,
                                                                target,
                                                                test_size=0.2,
                                                                random_state=0,
                                                                stratify=target)
x_train = train_dataset.drop('income', axis=1)
x_test = test_dataset.drop('income', axis=1)
print(train_dataset.head())

       age      workclass  educational-num marital-status     occupation  \
29101   19  Other/Unknown               10         Single  Other/Unknown   
10606   50        Private               10        Married    Blue-Collar   
9414    62        Private               10        Married   White-Collar   
1747    20        Private                9         Single   White-Collar   
10677   25        Private               13         Single   White-Collar   

        race  gender  hours-per-week  income  
29101  White  Female              30       0  
10606  White    Male              40       0  
9414   White    Male              40       0  
1747   White  Female              40       0  
10677  White  Female              40       0  


In [3]:
d = dice_ml.Data(dataframe=train_dataset, continuous_features=['age', 'educational-num','hours-per-week'], outcome_name='income')

In [4]:
model = MLP().to(device)
ckpt = torch.load('../'+args.model_path+f'MPL_{args.epoch}.pth', map_location='cpu')
model.load_state_dict(ckpt)
model.eval()


m = dice_ml.Model(model=model, backend='PYT',  func="ohe-min-max")
exp = dice_ml.Dice(d, m, method="gradient")

In [6]:
# generate counterfactuals
dice_exp = exp.generate_counterfactuals(x_test[1:3], total_CFs=4, desired_class="opposite")
# highlight only the changes
dice_exp.visualize_as_dataframe(show_only_changes=True)

 50%|█████     | 1/2 [00:26<00:26, 26.26s/it]

Diverse Counterfactuals found! total time taken: 00 min 25 sec


100%|██████████| 2/2 [00:31<00:00, 15.75s/it]

Diverse Counterfactuals found! total time taken: 00 min 05 sec
Query instance (original outcome : 1)


,age,workclass,educational-num,marital-status,occupation,race,gender,hours-per-week,income
0,46.0,Private,6.0,Married,Service,Asian-Pac-Islander,Male,37.0,0.708


...

,age,workclass,educational-num,marital-status,occupation,race,gender,hours-per-week,income
0,-,-,3.0,Single,-,Amer-Indian-Eskimo,-,-,0
1,-,-,-,Single,-,-,-,47.0,0
2,-,-,-,Divorced,-,-,-,31.0,0
3,-,-,-,-,-,-,Female,-,0


Query instance (original outcome : 0)


,age,workclass,educational-num,marital-status,occupation,race,gender,hours-per-week,income
0,47.0,Private,10.0,Divorced,White-Collar,White,Female,48.0,0.447



Diverse Counterfactual set (new outcome: 1.0)


,age,workclass,educational-num,marital-status,occupation,race,gender,hours-per-week,income
0,61.0,-,-,-,-,-,Male,43.0,1
1,40.0,-,8.0,Married,-,-,-,-,1
2,-,-,-,Married,-,-,-,58.0,1
3,-,-,14.0,-,-,-,-,-,1
